### Attempt 1 - using `pycorenlp` to access the java CoreNLP server (failed)

In [1]:
from pycorenlp import StanfordCoreNLP

In [21]:
nlp = StanfordCoreNLP('http://localhost:9090')

In [6]:
with open('./vec2world-news-sample.txt', 'r+') as f:
    news_sample = f.read().splitlines() 

    # to avoid the '\n' yield from f.readlines

In [8]:
news_sample[:1]

['现如今随着各大赛区2019年新赛季的接近，各位职业选手为了尽快调整好自己的状态也是纷纷开始了自己的冲分之旅，比如像SN的新打野选手就冲到了韩服第二名，而像rookie、jackeylove等上分如喝水选手也是重回韩服高分段，可是就在这种大背景下，一名职业选手最近却是遨游黑铁段位，甚至还遭到了队友的无情嘲讽，这到底是怎么一回事呢？事情的主人公还是我们的锅老师，由于新赛季段位重置的缘故，mlxg在国服的账号经过了九胜一负的排位赛之后竟然只定位到了黑铁四，其实作为一名职业选手来说，mlxg原本在韩服的排位分数并不出众，“钻石打野王”的称号也不是空穴来风。但是谁曾想到，mlxg竟然还有沦落到黑铁段位的时候。不过mlxg对于自己这个段位还是非常看得开的，在每局游戏开始的时候还会非常调皮的发一些能够让所有人都看到的骚话，比如什么“大家好，请叫我黑铁打野王”。不过很明显，并不是所有的队友都会给mlxg好脸色，比如下面的这名玩家。在某局游戏的开始，mlxg再次非常调皮的发送了一句“各位大哥大姐，求求你们带我上个青铜吧”，可是这时一名队友却突然毫不客气的回复道：“爷不想带你”。确实，由于mlxg隐藏分太高的缘故，这几天匹配到的队友都是钻石的“大神”，对于一名钻石段位的玩家来说，去带一名黑铁的萌新的确是一件既丢份又浪费时间的事情。不过虽然被队友怼了，但是mlxg的做法却是非常的正确，既然你不想带我，那么就让我来带你吧。最终在mlxg的carry下，这名回怼mlxg的玩家也是几乎以躺赢的姿态结束了这局游戏，可能这名玩家现在也正在为自己与一位真正大神发生矛盾而后悔不已吧。而经过自己的努力，mlxg在这两天也是终于晋级青铜分段，看来“黑铁打野王”的称号已经不适用了啊，各位同学，你们觉得青铜段位的锅老师应该叫什么样的外号呢？']

In [9]:
news_content = ''.join(news_sample)

In [13]:
news_content[:200]

'现如今随着各大赛区2019年新赛季的接近，各位职业选手为了尽快调整好自己的状态也是纷纷开始了自己的冲分之旅，比如像SN的新打野选手就冲到了韩服第二名，而像rookie、jackeylove等上分如喝水选手也是重回韩服高分段，可是就在这种大背景下，一名职业选手最近却是遨游黑铁段位，甚至还遭到了队友的无情嘲讽，这到底是怎么一回事呢？事情的主人公还是我们的锅老师，由于新赛季段位重置的缘故，mlxg在国服'

In [22]:
results = nlp.annotate(news_content,
                       properties={
                       'annotators': 'tokenize,ssplit,pos,lemma,ner,parse,depparse,coref,quote',
                       'outputFormat': 'json',
                       'timeout': 100000,
                   })

In [23]:
results[:100]

'CoreNLP request timed out. Your document may be too long.'

As an alternative to accessing the java CoreNLP server from a python API, we can directly call the server from the folder where `stanford-corenlp` is stored:

```bash
java -Xmx16G -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -props StanfordCoreNLP-chinese.properties -annotators tokenize,ssplit,pos,lemma,ner,parse,depparse,coref,quote -file /Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/nlp-project01/vec2world-news-sample.txt  -outputFormat xml -outputDirectory /Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/nlp-project01/
```

A list of other python APIs that I tried and failed (see `03-quotes-extraction-test1.py` file in this folder):
* 'stanfordcorenlp' wrapper developed by Lynten Guo -- see github page: https://github.com/Lynten/stanford-corenlp  (note -- this requires us to run code as root!)
* 'stanford-corenlp' wrapper developed by stanfordnlp -- see github page: https://github.com/stanfordnlp/python-stanford-corenlp
* the official StanfordNLP python package: https://github.com/stanfordnlp/stanfordnlp -- note: this package offer two types of utilities, the first one is performing basic NLP tasks (e.g, tokenize, lemma, pos, depparse) with the pre-trained neural pipeline directly from python (which requires `torch` > `1.0.0`); and the second one is to access the java CoreNLP server (similar to other python APIs mentioned above), which provides additional functions (e.g., tokensreges) other than the basic NLP tasks. However, the second one failed ... (see `03c-quote-extraction-test3.py`)

### Attempt 2 - using the python pipeline from `stanfordnlp`

Here we use the pre-trained neural pipeline from `stanfordnlp`, and before that, we need to download the chinese model

```python
import stanfordnlp
stanfordnlp.download('zh')
```

In [1]:
import stanfordnlp

In [2]:
pipeline = stanfordnlp.Pipeline(lang='zh')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/xinweixu/stanfordnlp_resources/zh_gsd_models/zh_gsd_tokenizer.pt', 'lang': 'zh', 'shorthand': 'zh_gsd', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/xinweixu/stanfordnlp_resources/zh_gsd_models/zh_gsd_tagger.pt', 'pretrain_path': '/Users/xinweixu/stanfordnlp_resources/zh_gsd_models/zh_gsd.pretrain.pt', 'lang': 'zh', 'shorthand': 'zh_gsd', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/xinweixu/stanfordnlp_resources/zh_gsd_models/zh_gsd_lemmatizer.pt', 'lang': 'zh', 'shorthand': 'zh_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/xinweixu/stanfordnlp_resources/zh_gsd_models/zh_gsd_parser.pt', 'pretrain_path': '/Users/xinweixu/stanfordnlp_resources

In [3]:
# read in the news sample

with open('./test_chinese_news.txt', encoding='utf-8') as f:
    news_sample = f.read().splitlines()   

In [4]:
news_sample

['美国贸易代表莱特希泽星期五表示，他已收到总统特朗普的指示，对其余尚未加征关税的所有中国商品加征关税，星期一将公布细节。此后特朗普发推特呼吁中国现在就采取行动(向美方让步)。关于中美下一步何时磋商，美财政部长姆努钦对媒体回应说：“目前还没有这方面的安排。” 美方是挑起贸易战的一方，自恃实力强大，相信它的关税大棒足以压垮中国。中方的回应则是太极哲学的示范版，坚守原则，不主动打也不惧战，用更大的承受力瓦解对方的攻击力，增加其他情况出现的可能性。美方攻势猛烈，但这种攻势在经济学上充满了非理性，对美国经济将导致常识性的自伤。美方显然寄希望于人类贸易史上从未有过的凶猛关税战可以一举摧毁中国的意志，以短时间内的损失换取中国接受不平等协议，铸就美国现政府的“辉煌大业”。美国的做法是赌博。美方不断发声，一个动作接一个动作，表现出的其实是他们急切希望这一切迅速生效的焦虑。']

In [5]:
# change list into string

news_content = ' '.join(news_sample)

In [6]:
news_content

'美国贸易代表莱特希泽星期五表示，他已收到总统特朗普的指示，对其余尚未加征关税的所有中国商品加征关税，星期一将公布细节。此后特朗普发推特呼吁中国现在就采取行动(向美方让步)。关于中美下一步何时磋商，美财政部长姆努钦对媒体回应说：“目前还没有这方面的安排。” 美方是挑起贸易战的一方，自恃实力强大，相信它的关税大棒足以压垮中国。中方的回应则是太极哲学的示范版，坚守原则，不主动打也不惧战，用更大的承受力瓦解对方的攻击力，增加其他情况出现的可能性。美方攻势猛烈，但这种攻势在经济学上充满了非理性，对美国经济将导致常识性的自伤。美方显然寄希望于人类贸易史上从未有过的凶猛关税战可以一举摧毁中国的意志，以短时间内的损失换取中国接受不平等协议，铸就美国现政府的“辉煌大业”。美国的做法是赌博。美方不断发声，一个动作接一个动作，表现出的其实是他们急切希望这一切迅速生效的焦虑。'

In [7]:
results = pipeline(news_content)

In [8]:
results.sentences[0].print_tokens()

<Token index=1;words=[<Word index=1;text=美国;lemma=美国;upos=PROPN;xpos=NNP;feats=_;governor=8;dependency_relation=nsubj>]>
<Token index=2;words=[<Word index=2;text=贸易;lemma=贸易;upos=ADV;xpos=RB;feats=_;governor=3;dependency_relation=advmod>]>
<Token index=3;words=[<Word index=3;text=代表;lemma=代表;upos=VERB;xpos=VV;feats=_;governor=6;dependency_relation=case>]>
<Token index=4;words=[<Word index=4;text=莱特希;lemma=莱特希;upos=PROPN;xpos=NNP;feats=_;governor=6;dependency_relation=nmod>]>
<Token index=5;words=[<Word index=5;text=泽;lemma=泽;upos=PROPN;xpos=NNP;feats=_;governor=6;dependency_relation=nmod>]>
<Token index=6;words=[<Word index=6;text=星期;lemma=星期;upos=NOUN;xpos=NN;feats=_;governor=8;dependency_relation=nsubj>]>
<Token index=7;words=[<Word index=7;text=五;lemma=五;upos=NUM;xpos=CD;feats=NumType=Card;governor=6;dependency_relation=nummod>]>
<Token index=8;words=[<Word index=8;text=表示;lemma=表示;upos=VERB;xpos=VV;feats=_;governor=0;dependency_relation=root>]>
<Token index=9;words=[<Word index=9;t

In [9]:
results.sentences

In [10]:
results.sentences[0].print_dependencies()

('美国', '8', 'nsubj')
('贸易', '3', 'advmod')
('代表', '6', 'case')
('莱特希', '6', 'nmod')
('泽', '6', 'nmod')
('星期', '8', 'nsubj')
('五', '6', 'nummod')
('表示', '0', 'root')
('，', '8', 'xcomp')
('他', '12', 'nsubj')
('已', '12', 'advmod')
('收到', '152', 'dep')
('总统', '14', 'appos')
('特朗普', '16', 'det')
('的', '14', 'case:dec')
('指示', '12', 'obj')
('，', '18', 'punct')
('对', '22', 'acl')
('其余', '22', 'nsubj')
('尚', '21', 'advmod')
('未', '22', 'advmod')
('加征', '27', 'acl:relcl')
('关税', '27', 'acl:relcl')
('的', '23', 'mark:relcl')
('所有', '27', 'det')
('中国', '27', 'nmod')
('商品', '28', 'nsubj')
('加征', '152', 'dep')
('关', '28', 'xcomp')
('税', '35', 'nsubj')
('，', '32', 'case')
('星期', '35', 'obl')
('一', '34', 'nummod')
('将', '32', 'acl')
('公布', '29', 'ccomp')
('细节', '35', 'obj')
('。', '152', 'punct')
('此后', '40', 'nmod')
('特朗普发', '40', 'nmod')
('推特', '41', 'nsubj')
('呼吁', '152', 'dep')
('中', '43', 'nmod')
('国', '44', 'nmod')
('现', '48', 'nmod')
('在', '52', 'acl')
('就采', '48', 'nmod')
('取行', '48', 'case:suf

In [11]:
results.sentences[0].words

[<Word index=1;text=美国;lemma=美国;upos=PROPN;xpos=NNP;feats=_;governor=8;dependency_relation=nsubj>,
 <Word index=2;text=贸易;lemma=贸易;upos=ADV;xpos=RB;feats=_;governor=3;dependency_relation=advmod>,
 <Word index=3;text=代表;lemma=代表;upos=VERB;xpos=VV;feats=_;governor=6;dependency_relation=case>,
 <Word index=4;text=莱特希;lemma=莱特希;upos=PROPN;xpos=NNP;feats=_;governor=6;dependency_relation=nmod>,
 <Word index=5;text=泽;lemma=泽;upos=PROPN;xpos=NNP;feats=_;governor=6;dependency_relation=nmod>,
 <Word index=6;text=星期;lemma=星期;upos=NOUN;xpos=NN;feats=_;governor=8;dependency_relation=nsubj>,
 <Word index=7;text=五;lemma=五;upos=NUM;xpos=CD;feats=NumType=Card;governor=6;dependency_relation=nummod>,
 <Word index=8;text=表示;lemma=表示;upos=VERB;xpos=VV;feats=_;governor=0;dependency_relation=root>,
 <Word index=9;text=，;lemma=，;upos=PUNCT;xpos=,;feats=_;governor=8;dependency_relation=xcomp>,
 <Word index=10;text=他;lemma=他;upos=PRON;xpos=PRP;feats=Person=3;governor=12;dependency_relation=nsubj>,
 <Word index=

In [12]:
for sent in results.sentences:
    for word in sent.words:
        print(word.text, word.upos, word.xpos)

美国 PROPN NNP
贸易 ADV RB
代表 VERB VV
莱特希 PROPN NNP
泽 PROPN NNP
星期 NOUN NN
五 NUM CD
表示 VERB VV
， PUNCT ,
他 PRON PRP
已 ADV RB
收到 VERB VV
总统 NOUN NN
特朗普 PROPN NNP
的 PART DEC
指示 NOUN NN
， PUNCT ,
对 VERB VV
其余 NOUN NN
尚 ADV RB
未 ADV RB
加征 VERB VV
关税 ADJ JJ
的 PART DEC
所有 DET DT
中国 NOUN NN
商品 NOUN NN
加征 VERB VV
关 VERB VV
税 PROPN NNP
， ADP IN
星期 NOUN NN
一 NUM CD
将 NOUN NN
公布 VERB VV
细节 NOUN NN
。 PUNCT ,
此后 NOUN NN
特朗普发 PROPN NNP
推特 PROPN NNP
呼吁 VERB VV
中 PROPN NNP
国 PROPN NNP
现 PROPN NNP
在 VERB VV
就采 PROPN NNP
取行 VERB VV
动 PART SFN
( PUNCT (
向 ADP IN
美方 PROPN NNP
让步 VERB VV
) PUNCT )
。 ADV RB
关 PROPN NNP
于 PROPN NNP
中 PROPN NNP
美下 PROPN NNP
一 NUM CD
步 NOUN NNB
何 PROPN NNP
时 PROPN NNP
磋 VERB VV
商 NOUN NN
， PUNCT ,
美财 PROPN NNP
政部 NOUN NN
长姆努 PROPN NNP
钦对 VERB VV
媒体 NOUN NN
回应 VERB VV
说： VERB VV
“ PUNCT ,
目前 NOUN NN
还没 ADV RB
有 VERB VV
这 DET DT
方面 NOUN NN
的 PART DEC
安排 NOUN NN
。 PUNCT ,
” ADV RB
美方 NOUN NN
是 VERB VC
挑起 VERB VV
贸易 NOUN NN
战 PART SFN
的 PART DEC
一 NUM CD
方 NOUN NN
， PUNCT ,
自恃 NOUN NN

In [13]:
# get a prettier print

print(*[f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\txpos: {word.xpos}' for sent in results.sentences for word in sent.words], sep='\n')

# code taken from:
# https://stanfordnlp.github.io/stanfordnlp/pipeline.html

text: 美国 	lemma: 美国	upos: PROPN	xpos: NNP
text: 贸易 	lemma: 贸易	upos: ADV	xpos: RB
text: 代表 	lemma: 代表	upos: VERB	xpos: VV
text: 莱特希 	lemma: 莱特希	upos: PROPN	xpos: NNP
text: 泽 	lemma: 泽	upos: PROPN	xpos: NNP
text: 星期 	lemma: 星期	upos: NOUN	xpos: NN
text: 五 	lemma: 五	upos: NUM	xpos: CD
text: 表示 	lemma: 表示	upos: VERB	xpos: VV
text: ， 	lemma: ，	upos: PUNCT	xpos: ,
text: 他 	lemma: 他	upos: PRON	xpos: PRP
text: 已 	lemma: 已	upos: ADV	xpos: RB
text: 收到 	lemma: 收到	upos: VERB	xpos: VV
text: 总统 	lemma: 总统	upos: NOUN	xpos: NN
text: 特朗普 	lemma: 特朗普	upos: PROPN	xpos: NNP
text: 的 	lemma: 的	upos: PART	xpos: DEC
text: 指示 	lemma: 指示	upos: NOUN	xpos: NN
text: ， 	lemma: ，	upos: PUNCT	xpos: ,
text: 对 	lemma: 对	upos: VERB	xpos: VV
text: 其余 	lemma: 其余	upos: NOUN	xpos: NN
text: 尚 	lemma: 尚	upos: ADV	xpos: RB
text: 未 	lemma: 未	upos: ADV	xpos: RB
text: 加征 	lemma: 加征	upos: VERB	xpos: VV
text: 关税 	lemma: 关税	upos: ADJ	xpos: JJ
text: 的 	lemma: 的	upos: PART	xpos: DEC
text: 所有 	lemma: 所有	upos: DET	xpos: DT
text: 中国 	lemm

Now let's try looking for `word.lemma` = "说", and `word.upos` = `VERB`

In [40]:
# import list of "说"

with open("./saying_verbs_cleaned.txt", "r+") as f:
    saying_verbs = f.read().splitlines()    

In [15]:
# find similar words to "说" in the text

for sent in results.sentences:
    for word in sent.words:
        if word.text in saying_verbs and word.upos == 'VERB':
            print(word.index, word.text, word.upos)
            print(''.join(t.text for t in sent.tokens))

8 表示 VERB
美国贸易代表莱特希泽星期五表示，他已收到总统特朗普的指示，对其余尚未加征关税的所有中国商品加征关税，星期一将公布细节。此后特朗普发推特呼吁中国现在就采取行动(向美方让步)。关于中美下一步何时磋商，美财政部长姆努钦对媒体回应说：“目前还没有这方面的安排。”美方是挑起贸易战的一方，自恃实力强大，相信它的关税大棒足以压垮中国。中方的回应则是太极哲学的示范版，坚守原则，不主动打也不惧战，用更大的承受力瓦解对方的攻击力，增加其他情况出现的可能性。美方攻势猛烈，但这种攻势在经济学上充满了非理性，对美国经济将导致常识性的自伤。
41 呼吁 VERB
美国贸易代表莱特希泽星期五表示，他已收到总统特朗普的指示，对其余尚未加征关税的所有中国商品加征关税，星期一将公布细节。此后特朗普发推特呼吁中国现在就采取行动(向美方让步)。关于中美下一步何时磋商，美财政部长姆努钦对媒体回应说：“目前还没有这方面的安排。”美方是挑起贸易战的一方，自恃实力强大，相信它的关税大棒足以压垮中国。中方的回应则是太极哲学的示范版，坚守原则，不主动打也不惧战，用更大的承受力瓦解对方的攻击力，增加其他情况出现的可能性。美方攻势猛烈，但这种攻势在经济学上充满了非理性，对美国经济将导致常识性的自伤。


In [16]:
# note here that "说:" cannot be identified due to the punctuation mark,
# so we need to strip the punctuation marks 
# before comparing the words with list of saying_verbs

import re

def find_token(string):
    """Define a procedure to isolate all words in the text, so as to 
    -- remove irrelevant strings '\r'and '\n';
    -- remove all punctuation marks
    """
    return ' '.join(re.findall('[\w|\d]+', string))

In [31]:
for sent in results.sentences:
    for word in sent.words:
        if find_token(word.text) in saying_verbs and word.upos == 'VERB':
            print(word.index, word.text, word.upos)
            index = int(word.index) # print the text before the verb
            print(''.join(t.text for t in sent.tokens[:index]))

8 表示 VERB
美国贸易代表莱特希泽星期五表示
41 呼吁 VERB
美国贸易代表莱特希泽星期五表示，他已收到总统特朗普的指示，对其余尚未加征关税的所有中国商品加征关税，星期一将公布细节。此后特朗普发推特呼吁
72 说： VERB
美国贸易代表莱特希泽星期五表示，他已收到总统特朗普的指示，对其余尚未加征关税的所有中国商品加征关税，星期一将公布细节。此后特朗普发推特呼吁中国现在就采取行动(向美方让步)。关于中美下一步何时磋商，美财政部长姆努钦对媒体回应说：


In [29]:
# note that the index is a string, so we need to turn it into an integer
type(results.sentences[0].tokens[18].index)

str

In [33]:
# try another news sample

with open('./vec2world-news-sample.txt', encoding='utf-8') as f:
    news_sample_2 = f.read().splitlines()   

In [34]:
news_content_2 = ' '.join(news_sample_2)

In [35]:
results_2 = pipeline(news_content_2)

In [41]:
for sent in results_2.sentences:
    for word in sent.words:
        if find_token(word.text) in saying_verbs and word.upos == 'VERB':
            print(word.index, word.text, word.upos, word.dependency_relation)
            index = int(word.index) # print the text before the verb
            print(''.join(t.text for t in sent.tokens[:index]))

41 回复 VERB case:suff
在某局游戏的开始，mlxg再次非常调皮的发送了一句“各位大哥大姐，求求你们带我上个青铜吧”，可是这时一名队友却突然毫不客气的回复
84 说 VERB acl
在某局游戏的开始，mlxg再次非常调皮的发送了一句“各位大哥大姐，求求你们带我上个青铜吧”，可是这时一名队友却突然毫不客气的回复道：“爷不想带你”。确实，由于mlxg隐藏分太高的缘故，这几天匹配到的队友都是钻石的“大神”，对于一名钻石段位的玩家来说
12 说 VERB ccomp
特别是Uzi，这两天在峡谷之巅的排位可以说
21 透露 VERB acl
他马上要参加虎牙星盛典的LOL全明星赛！不过今天Uzi直播的时候，无意间透露
28 表示 VERB dep
他马上要参加虎牙星盛典的LOL全明星赛！不过今天Uzi直播的时候，无意间透露了他的按键习惯，表示
140 说， VERB csubj
他马上要参加虎牙星盛典的LOL全明星赛！不过今天Uzi直播的时候，无意间透露了他的按键习惯，表示自己的用中指按A键！这个消息出来后，不少玩家都有些懵逼，A键原来还能这样玩的？还有网友调侃道：这样岂不是Uzi每次他A人的时候，都是在朝人竖中指，哈哈哈哈。不得不说，这个确实是挺奇葩的，大多数应该用的是无名指吧，少数用小拇指，中指的更是少之又少！于是不少玩家就会发现，其实很多职业选手在键位设置下，也是一个比一个奇葩！Uzi的键位设置相对来说，
90 解释 VERB ccomp
根据网友的测试，原来MLXG是利用快捷施法和智能施法这两个的功能将面板的键位隐藏了起来，估计是MLXG不想让别人看到他的键位习惯吧！于是也有玩家调侃道：这职业选手的键位设置，真的是一个比一个奇葩啊！看完正常的Uzi键位后，再来看看马哥的键位设置，你会发现马哥也是会玩！不过根据马哥粉丝解释
10 问 VERB case:suff
而拳头似乎也意识到了这个问
34 说 VERB dep
下面，村长就来给大家盘点一下在2018年新出的皮肤当中，最不受欢迎的几款皮肤。睡衣守护者露露这款皮肤可以说
124 说 VERB dep
下面，村长就来给大家盘点一下在2018年新出的皮肤当中，最不受欢迎的几款皮肤。睡衣守护者露露这款皮肤可以说是完完全全的照骗"！看着原画上的露露非常温柔可爱。但是进入游戏之后，会发现这和原画相比根本就不是

In [44]:
# if we also filter out verbs that are not the 'root'

for sent in results_2.sentences:
    for word in sent.words:
        if find_token(word.text) in saying_verbs:
            if word.upos == 'VERB' and word.dependency_relation == 'root':
                print(word.index, word.text, word.upos, word.dependency_relation)
                # print the all tokens in the sentence containing the verb
                print(''.join(t.text for t in sent.tokens))

35 说 VERB root
大玩家还是愿意买单的。毕竟以往的神作都摆在那里，谁都会卖个面子。游戏王：决斗链接对于90后男性玩家来说，这个应该是绝对的童年回忆，
32 表示 VERB root
……可以说是嘲讽满满了！最近，很快就要发售的《生化危机2：重制版》公布了一张女丧尸的人设图，并表示玩家们会看到一些熟悉的面孔。有网友猜测可能女丧尸的脸是借用了游戏开发人员的长相？但有网友脑洞更大：这个女丧尸的脸……张学友的“吔屎
5 觉得 VERB root
啦”？Emmmmm雕妹觉得，还真是挺像的哈！在欧洲文艺复兴时期诞生了许许多多的怪人不对也可以叫作怪才有那么一个人天生爱倒腾各种稀奇古怪的东西造过滑翔器造过风速计造过陀螺仪对医学感兴趣描画人体骨骼图画成为一个解剖大师第一个敢跟宗教sayno的人提出实践验证真理给后世一大群科学家铺垫发明创造之路也喜欢设计各种军事机械和武器于是成为我们游戏玩家的老熟人刺客信条提供各种武器火药的就是这家伙甚至就连画个鸡蛋都能出现在小学生的语文课本本来这种天才型的头号玩家应该是跟爱迪生一样作为发明家和科学家被我们写进史册的可偏偏最让他流芳百世的是他闲时下来的爱好说到这里你们应该大概能猜到他是谁了吧他最巅峰出名的作品就是收藏在法国卢浮宫博物馆的油画蒙娜丽莎的微笑而他就是那个被称作人类历史上绝无仅有全才的男人列奥纳多·迪·皮耶罗·达·芬奇很多健身玩家，本身也可能是游戏玩家，


In [45]:
!pwd

/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/nlp-AI-course/Project01-ExtractingKeywords


In [46]:
# try the full news sample!
file_path = '/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/nlp-project01/vec2world-news-original.txt'
with open(file_path, encoding='utf-8') as f:
    news = f.read().splitlines()  

In [48]:
news[:3]

['【图集简介】 由玩家shishioh带来的真骨雕假面骑士亚马逊的评测图集，这款真骨雕假面骑士亚马逊以通贩的形式贩售，配件不算丰富，只有8个手型、2个披风替换件和2个腰带替换件。图片转自shishioh，侵删。',
 '现如今随着各大赛区2019年新赛季的接近，各位职业选手为了尽快调整好自己的状态也是纷纷开始了自己的冲分之旅，比如像SN的新打野选手就冲到了韩服第二名，而像rookie、jackeylove等上分如喝水选手也是重回韩服高分段，可是就在这种大背景下，一名职业选手最近却是遨游黑铁段位，甚至还遭到了队友的无情嘲讽，这到底是怎么一回事呢？事情的主人公还是我们的锅老师，由于新赛季段位重置的缘故，mlxg在国服的账号经过了九胜一负的排位赛之后竟然只定位到了黑铁四，其实作为一名职业选手来说，mlxg原本在韩服的排位分数并不出众，“钻石打野王”的称号也不是空穴来风。但是谁曾想到，mlxg竟然还有沦落到黑铁段位的时候。不过mlxg对于自己这个段位还是非常看得开的，在每局游戏开始的时候还会非常调皮的发一些能够让所有人都看到的骚话，比如什么“大家好，请叫我黑铁打野王”。不过很明显，并不是所有的队友都会给mlxg好脸色，比如下面的这名玩家。在某局游戏的开始，mlxg再次非常调皮的发送了一句“各位大哥大姐，求求你们带我上个青铜吧”，可是这时一名队友却突然毫不客气的回复道：“爷不想带你”。确实，由于mlxg隐藏分太高的缘故，这几天匹配到的队友都是钻石的“大神”，对于一名钻石段位的玩家来说，去带一名黑铁的萌新的确是一件既丢份又浪费时间的事情。不过虽然被队友怼了，但是mlxg的做法却是非常的正确，既然你不想带我，那么就让我来带你吧。最终在mlxg的carry下，这名回怼mlxg的玩家也是几乎以躺赢的姿态结束了这局游戏，可能这名玩家现在也正在为自己与一位真正大神发生矛盾而后悔不已吧。而经过自己的努力，mlxg在这两天也是终于晋级青铜分段，看来“黑铁打野王”的称号已经不适用了啊，各位同学，你们觉得青铜段位的锅老师应该叫什么样的外号呢？',
 '引言：Rookie撞脸肖央？当dopa和小沈阳同框出现，网友笑了！作为知名的英雄联盟职业选手，rookie等了这个全球总冠军很久，而在这次的s8全球总决赛上，IG终于拿到了LPL赛区八年来的第一个冠军。拿到冠军后，网友却突然发现rookie为什么

In [49]:
len(news)

191698

In [50]:
news_content = ' '.join(news)

In [ ]:
news_parsed = pipeline(news_content)